# Etapa de pré-processamento

Importação das bibliotecas

In [1]:
import pandas as pd
from datetime import datetime

Carregar o arquivo

In [2]:
file_path = "/content/DESAFIO ETAPA 1.xlsx"
df = pd.read_excel(file_path)

Padronização de datas (YYYY-MM-DD)

In [3]:
def converter_data(data_str):
        try:
            # Remover espaços em branco e converter para string
            data_str = str(data_str).strip()

            # Lista de formatos de data para tentar
            formatos = [
                '%d/%m/%Y',          # 27/03/2023
                '%Y/%m/%d',          # 2023/01/08
                '%d/%m/%y',          # 10/05/23
                '%Y-%m-%d',          # 2023-05-30
                '%d-%m-%Y',          # 16-03-2023
                '%Y-%m-%d %H:%M:%S', # 2024-04-19 00:00:00
                '%d-%m-%y'           # 01-03-23
            ]

            for fmt in formatos:
                try:
                    # Tentar converter para datetime
                    data = datetime.strptime(data_str, fmt)
                    # Retornar no formato YYYY-MM-DD
                    return data.strftime('%Y-%m-%d')
                except ValueError:
                    continue

            # Se nenhum formato funcionar, retornar a data original
            return data_str
        except:
            # Em caso de erro, retornar a data original
            return data_str

In [4]:
# Aplicar a conversão à coluna "Data Venda"
df['Data Venda'] = df['Data Venda'].astype(str).apply(converter_data)

Padronização dos campos de texto e código do cliente

In [5]:
'''
df["Status Pagamento"] = df["Status Pagamento"].astype(str).str.strip().str.upper()
'''
colunas_texto = ["Status Pagamento", "Região", "Produto", "Representante", "Código Cliente"]
for coluna in colunas_texto:
    df[coluna] = df[coluna].astype(str).str.strip().str.upper()

Conversão correta dos campos de valores (Valor Venda, Comissão) para numérico (float)

In [6]:
def limpar_valor(v):
    if pd.isna(v):
        return np.nan
    v = str(v)
    v = v.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".").replace("%", "")
    try:
        return float(v)
    except:
        return np.nan

In [7]:
# Aplicar a conversão à coluna "Data Venda"
df["Valor Venda"] = df["Valor Venda"].apply(limpar_valor)
df["Comissão"] = df["Comissão"].apply(limpar_valor)

  # Se comissão > 1, é percentual e deve ser convertido para decimal
df.loc[df["Comissão"] > 1, "Comissão"] = df["Comissão"] / 100

Calcular a coluna Valor Comissão (Valor Venda × Comissão)

In [8]:
df["Valor Comissão"] = df["Valor Venda"] * df["Comissão"]

Eliminar duplicidades (ex: vendas idênticas por engano)

In [9]:
df = df.drop_duplicates()

In [10]:
#Visualizar resultado final
print(df.head())

   Data Venda Representante        Região     Produto  Valor Venda  Comissão  \
0  2023-03-27    JOAO SILVA       SUDESTE     CELULAR       2550.0      0.10   
1  2023-01-08      M. SOUZA           SUL    NOTEBOOK       4679.0      0.10   
2  2023-05-30    JOÃO SILVA         NORTE  IMPRESSORA       1530.0      0.08   
3  2023-05-10      M. SOUZA  CENTRO-OESTE     CELULAR     166600.0      0.08   
4  2023-03-28     ANA PAULA       SUDESTE    NOTEBOOK       1265.0      0.10   

  Status Pagamento Código Cliente  Valor Comissão  
0             PAGO           A594           255.0  
1        EM ABERTO           E121           467.9  
2        EM ABERTO           A342           122.4  
3             PAGO           E925         13328.0  
4             PAGO           D888           126.5  


Exportação do arquivo tratado

In [11]:
df.to_excel("dados_tratados.xlsx", index=False)
print("Arquivo 'dados_tratados.xlsx' salvo com sucesso!")

Arquivo 'dados_tratados.xlsx' salvo com sucesso!


# ANÁLISE DE DADOS
- Podendo ser em Python, SQL ou Excel.
- Total de vendas por representante
- Total de comissão por representante
- Ticket médio por produto (Valor Total de Vendas / No de Vendas)
- Identificação de compras duplicadas (mesmo cliente, valor, data e produto)
- Contagem de vendas por status de pagamento
- Análise do total de vendas por região


In [12]:
def analisar_vendas(caminho_arquivo, arquivo_saida="analise_vendas.xlsx"):
    # 1. Ler dados
    df = pd.read_excel(caminho_arquivo)

    # 2. Garantir coluna "Valor Comissão"
    if "Valor Comissão" not in df.columns:
        df["Valor Comissão"] = df["Valor Venda"] * df["Comissão"]

    # 3. Total de vendas por representante
    total_vendas_rep = df.groupby("Representante", as_index=False)["Valor Venda"].sum().rename(columns={"Valor Venda": "Total Vendas"})

    # 4. Total de comissão por representante
    total_comissao_rep = df.groupby("Representante", as_index=False)["Valor Comissão"].sum().rename(columns={"Valor Comissão": "Total Comissão"})

    # 5. Ticket médio por produto
    ticket_medio_prod = df.groupby("Produto", as_index=False)["Valor Venda"].mean().rename(columns={"Valor Venda": "Ticket Médio"})

    # 6. Compras duplicadas
    compras_duplicadas = df[df.duplicated(subset=["Código Cliente", "Valor Venda", "Data Venda", "Produto"], keep=False)]

    # 7. Contagem de vendas por status de pagamento
    contagem_status = df["Status Pagamento"].value_counts().reset_index()
    contagem_status.columns = ["Status Pagamento", "Quantidade"]

    # 8. Total de vendas por região
    total_vendas_regiao = df.groupby("Região", as_index=False)["Valor Venda"].sum().rename(columns={"Valor Venda": "Total Vendas Região"})

    # 9. Exportar para Excel com abas separadas
    with pd.ExcelWriter(arquivo_saida) as writer:
        for df, sheet in [
            (total_vendas_rep, "Total Vendas Representante"),
            (total_comissao_rep, "Total Comissão Representante"),
            (ticket_medio_prod, "Ticket Médio Produto"),
            (compras_duplicadas, "Compras Duplicadas"),
            (contagem_status, "Contagem Status"),
            (total_vendas_regiao, "Total Vendas Região")
        ]:
            df.to_excel(writer, sheet_name=sheet, index=False)

    print(f"Arquivo '{arquivo_saida}' gerado com sucesso!")

analisar_vendas("dados_tratados.xlsx", "analise_vendas.xlsx")

Arquivo 'analise_vendas.xlsx' gerado com sucesso!
